# General

In [ ]:
import numpy as np
import copy as cp

import energyflow as ef
from energyflow.archs import CNN
from energyflow.datasets import qg_jets
from energyflow.utils import data_split, pixelate, standardize, to_categorical, zero_center

from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.model_selection import train_test_split
import math
import matplotlib.pyplot as plt

import matplotlib
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

In [ ]:
## Used for ODU's HPC

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.config.experimental.list_physical_devices("GPU")


import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Data

In [ ]:
# load data 
X, y = qg_jets.load(num_data=100000)      # remove PID

X = X[:,:,:3]

# convert labels to categorical
Y = to_categorical(y, num_classes=2)

In [ ]:
# image parameters
R = 0.4
img_width = 2*R
npix = 64
nb_chan = 1               # just one channel
norm = True

# Data Preprocessing

In [ ]:
# preprocess by centering jets and normalizing pts
for x in X:
    mask = x[:,0] > 0                                                                # Takes the particles that have transverse momentum
    yphi_avg = np.average(x[mask,1:3], weights=x[mask,0], axis=0)                    # Takes the average of the rapidity and phi
    x[mask,1:3] -= yphi_avg                                                          # subtracting each rapidity and phi by the average
    x[mask,0] /= x[:,0].sum()                                                        # normalizing the transverse momentum

In [ ]:
# make jet images
images2 = np.asarray([pixelate(x, npix=npix, img_width=img_width, nb_chan=nb_chan, 
                                 charged_counts_only=False, norm=norm) for x in X])
# images = jet, phi, rapidty, pt

In [ ]:
images2.shape

In [ ]:
(X_train2, X_test2, Y_train2, Y_test2) = train_test_split(images2,Y, test_size=0.20, shuffle=True)

In [ ]:
X_train2.shape, X_test2.shape

## Jet Image Example

In [ ]:
%matplotlib inline
plt.figure(figsize=(15,10))
plt.imshow(X_test2[0,:,:], cmap = 'Blues')
plt.xlabel("Phi", fontsize = 15)
plt.ylabel("Rapidity", fontsize = 15)
plt.colorbar()
#plt.savefig("")

In [ ]:
zval_org = X_test2.flatten()

## Z-values

In [ ]:
%matplotlib inline
plt.hist(zval_org, bins = 30, color = 'b', histtype = 'step')
plt.xlabel("Z-values", fontsize = 12)
plt.ylabel("Count (log base 10 scaling)", fontsize = 12)
plt.semilogy()
#plt.savefig("")
plt.show()

In [ ]:
%matplotlib inline
bounds = [0]
zval_log = np.log10(zval_org[zval_org != 0])
plt.hist(zval_log, bins = 50, color = 'b', histtype = 'step')
plt.semilogy()
plt.xlabel("Z-values", fontsize = 12)
plt.ylabel("Count (log base 10 scaling)", fontsize = 12)
#plt.savefig("")
plt.show()

In [ ]:
%matplotlib inline
plt.hist(zval_log, bins = 50, color = 'b', histtype = 'step')
plt.xlabel("Z-values", fontsize = 12)
plt.ylabel("Count", fontsize = 12)
#plt.savefig("")
plt.show()

In [ ]:
phi = np.concatenate([np.nonzero(x)[0] for x in X_test2])
rapidity = np.concatenate([np.nonzero(x)[1] for x in X_test2])

## Phi-distribution

In [ ]:
%matplotlib inline
plt.hist(phi, bins = 20, color = 'b', histtype = 'step', label = 'Phi')
plt.hist(rapidity, bins = 20, color = 'r', histtype = 'step', label = 'Rapidity')
plt.semilogy()
plt.xlabel("Phi and Rapidity", fontsize = 12)
plt.ylabel("Count (log base 10 scaling)", fontsize = 12)
plt.legend()
#plt.savefig("")
plt.show()

## Image Transformation: imgs_trnsf

In [ ]:
imgs_trnsf = cp.deepcopy(images2)
imgs_trnsf[imgs_trnsf == 0] = 10
imgs_trnsf = np.log10(imgs_trnsf)
imgs_trnsf[imgs_trnsf == 1] = -8

In [ ]:
zval_trnsf = imgs_trnsf.flatten()
%matplotlib inline
plt.hist(zval_trnsf, bins = 50, color = 'b', histtype = 'step')
plt.semilogy()
plt.show()

# Standard Autoencoder: SAE

## Hyperparameters

In [ ]:
kernel = ###
lr = ###
activ = 'elu'
rate = ###

## Encoder

In [ ]:
x = layers.Conv2D(kernel, (3,3), activation = activ, padding = 'same')(input_img[0])
x = layers.BatchNormalization()(x)
x = layers.Dropout(rate)(x)

x = layers.Conv2D(kernel, (3,3), activation = activ, padding = 'same')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(rate)(x)

x = layers.AveragePooling2D(pool_size = (2,2), strides = (1,1), padding = 'same')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(rate)(x)

x = layers.Conv2D(kernel, (3,3), activation = activ, padding = 'same')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(rate)(x)

x = layers.Conv2D(kernel, (3,3), activation = activ, padding = 'same')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(rate)(x)

x = layers.AveragePooling2D(pool_size = (2,2), strides = (1,1), padding = 'same')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(rate)(x)

x = layers.Flatten()(x)

encoded64 = layers.Dense(units = 256, activation = activ)(x)

## Decoder

In [ ]:
x = layers.Dense(units = 256, activation = activ)(encoded64)
x = layers.BatchNormalization()(x)
x = layers.Dropout(rate)(x)

x = layers.Reshape(target_shape=(16,16, 1))(x)

x = layers.Conv2DTranspose(kernel, (3,3), strides = 2, padding = 'same', activation = activ)(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(rate)(x)

x = layers.Conv2D(kernel, (3,3), activation = activ, padding = 'same')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(rate)(x)

x = layers.Conv2D(kernel, (3,3), activation = activ, padding = 'same')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(rate)(x)

x = layers.Conv2DTranspose(kernel, (3,3), strides = 2, padding = 'same')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(rate)(x)

x = layers.Conv2D(kernel, (3,3), activation = activ, padding = 'same')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(rate)(x)

decoded64 = layers.Conv2D(1, (3,3), activation = 'sigmoid', padding = 'same')(x)

## Fitting

In [ ]:
autoencoder1 = keras.Model(input_img[0], decoded64)
#autoencoder2 = keras.Model(input_img[1], decoded64)
opt = keras.optimizers.Adam(learning_rate=lr)
autoencoder1.compile(optimizer = opt, loss='mean_squared_error', metrics=['accuracy'])

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 60)
history1 = autoencoder1.fit(x = images2, y = images2, epochs = 100, batch_size = 32, shuffle = True,
                                validation_split = 0.20, callbacks = [callback])

In [ ]:
def plotting(loss, epochs, color, label):
    ax = plt.subplot(2, 2, 1)
    plt.plot(loss[0], color[0], label = label[0])
    plt.plot(loss[1], color[1], label = label[1])
    plt.legend()
    plt.semilogy()
    plt.xlabel('Epoch')
    plt.ylabel('Loss')

In [ ]:
%matplotlib inline

# Get training and test loss histories
training_loss = [history1.history['loss'], history2.history['loss']]
val_loss = [history1.history['val_loss'], history2.history['val_loss']]

loss1 = [training_loss[0], val_loss[0]]
loss2 = [training_loss[1], val_loss[1]]

# Visualize loss history
plt.figure(figsize=(15,10))
colors = ['r--', 'b-']
labels = ['training','validation']
plotting(loss1, 100, colors, labels)
colors = ['g--', 'k-']
labels = ['training_trnsf','validation_trnsf']
plotting(loss2, 100, colors, labels)
#plt.savefig("")
plt.show()

# Keras Tuner

In [ ]:
def model(filters, rate, lr, activ_choice):
    
    input_img = keras.Input(shape = (64,64,1))
        # Encoding
    x = layers.Conv2D(filters, (3,3), activation = activ_choice, padding = 'same')(input_img)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(rate)(x)
    x = layers.Conv2D(filters, (3,3), activation = activ_choice, padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(rate)(x)
    x = layers.AveragePooling2D(pool_size = (2,2), strides = (1,1), padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(rate)(x)
    x = layers.Conv2D(filters, (3,3), activation = activ_choice, padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(rate)(x)
    x = layers.Conv2D(filters, (3,3), activation = activ_choice, padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(rate)(x)
    x = layers.AveragePooling2D(pool_size = (2,2), strides = (1,1), padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(rate)(x)
    x = layers.Flatten()(x)
    encoded = layers.Dense(units = 256, activation = activ_choice)(x)
    
        # Decoding
    x = layers.Dense(units = 256, activation = activ_choice)(encoded)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(rate)(x)
    x = layers.Reshape(target_shape=(16,16, 1))(x)
    x = layers.Conv2DTranspose(filters, (3,3), strides = 2, padding = 'same', activation = activ_choice)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(rate)(x)
    x = layers.Conv2D(filters, (3,3), activation = activ_choice, padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(rate)(x)
    x = layers.Conv2D(filters, (3,3), activation = activ_choice, padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(rate)(x)
    x = layers.Conv2DTranspose(filters, (3,3), strides = 2, padding = 'same', activation = activ_choice)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(rate)(x)
    x = layers.Conv2D(filters, (3,3), activation = activ_choice, padding = 'same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(rate)(x)
    decoded = layers.Conv2D(1, (3,3), activation = 'sigmoid', padding = 'same')(x)
    
    model = keras.Model(input_img, decoded)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="mean_squared_error",
        metrics=["accuracy"],
    )
    return model

In [ ]:
def AE_hps(hp):
    filters = hp.Int("filters", min_value = 2, max_value = 16, step = 4)
    rate = hp.Float("rate", min_value = 0.0, max_value = 0.30, step = 0.05)
    learning_rate = hp.Float("lr", min_value=1e-5, max_value=1e-2, sampling="log")
    choice = 'elu'
    
    hpmodel = model(filters = filters, rate = rate, lr = learning_rate, activ_choice = choice)
    return hpmodel

In [ ]:
AE_hps(kt.HyperParameters())

In [ ]:
tuner = kt.BayesianOptimization(hypermodel = AE_hps, objective = 'val_loss', max_trials = 100, 
                                num_initial_points=None, alpha=0.0001, 
                                beta=4, max_consecutive_failed_trials=2,
                               directory="bayesianopt", project_name="autoencoder")
    # beta - Exploration and exploitation. The larger it is, the more explorative it is.
    # alpha - Expected amount of noise

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10)
tuner.search(x = images2, y = images2, epochs = 100, batch_size = 32, shuffle = True,
                              validation_split = 0.20, callbacks = [callback])

# Variational Autoencoder: VAE

## Hyperparameters

In [ ]:
kernel = 16
lr = 0.00001
activ = 'elu'
rate = 0.05

In [ ]:
class sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        sample = z_mean + tf.exp(0.5 * z_log_var) * epsilon
        return sample

In [ ]:
encoder_input = keras.Input(shape = (images2.shape[1],images2.shape[1],1))

x = layers.Conv2D(kernel, (3,3), activation = activ, padding = 'same')(encoder_input)
x = layers.BatchNormalization()(x)
x = layers.Dropout(rate)(x)

x = layers.Conv2D(kernel, (3,3), activation = activ, padding = 'same')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(rate)(x)

x = layers.AveragePooling2D(pool_size = (2,2), strides = (1,1), padding = 'same')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(rate)(x)

x = layers.Conv2D(kernel, (3,3), activation = activ, padding = 'same')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(rate)(x)

x = layers.Conv2D(kernel, (3,3), activation = activ, padding = 'same')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(rate)(x)

x = layers.AveragePooling2D(pool_size = (2,2), strides = (1,1), padding = 'same')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(rate)(x)

x = layers.Flatten()(x)

x = layers.Dense(units = 256, activation = activ)(x)
encoded = layers.Dense(units = 64, activation = activ)(x)
z_mean = layers.Dense(64, name = 'mean')(encoded)
z_log_var = layers.Dense(64, name = 'log_var')(encoded)
z = sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_input, [z_mean, z_log_var, z], name = 'encoder')
encoder.summary()

In [ ]:
decoder_input = keras.Input(shape = (64,))

x = layers.Dense(units = 64, activation = activ)(decoder_input)
x = layers.BatchNormalization()(x)
x = layers.Dropout(rate)(x)

x = layers.Dense(units = 256, activation = activ)(decoder_input)
x = layers.BatchNormalization()(x)
x = layers.Dropout(rate)(x)

x = layers.Reshape(target_shape=(16,16, 1))(x)

x = layers.Conv2DTranspose(kernel, (3,3), strides = 2, padding = 'same', activation = activ)(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(rate)(x)

x = layers.Conv2D(kernel, (3,3), activation = activ, padding = 'same')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(rate)(x)

x = layers.Conv2D(kernel, (3,3), activation = activ, padding = 'same')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(rate)(x)

x = layers.Conv2DTranspose(kernel, (3,3), strides = 2, padding = 'same')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(rate)(x)

x = layers.Conv2D(kernel, (3,3), activation = activ, padding = 'same')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(rate)(x)

decoded = layers.Conv2D(1, (3,3), activation = 'sigmoid', padding = 'same')(x)
decoder = keras.Model(decoder_input, decoded, name = 'decoder')
decoder.summary()

In [ ]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.mean_squared_error(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

## Fitting

In [ ]:
opt = keras.optimizers.Adam(learning_rate=lr)
vae = VAE(encoder, decoder)
vae.compile(optimizer = opt)
callback = tf.keras.callbacks.EarlyStopping(monitor = 'kl_loss', patience = 20)
history = vae.fit(x = X_train2, epochs = 100, batch_size = 32, shuffle = True, callbacks = [callback])

In [ ]:
%matplotlib inline
training_loss = history.history['kl_loss']
plt.figure(figsize=(10,5))
colors = ['r--']
labels = ['training']
plt.plot(training_loss, colors[0], label = labels[0])
plt.legend()
plt.semilogy()
plt.xlabel('Epoch', fontsize = 12)
plt.ylabel('KL_Loss', fontsize = 12)
#plt.savefig("VAE_klloss.png")
plt.show()

# Predicting

## VAE

In [40]:
m, v, z = encoder.predict(X_test2)
decoded_imgs = decoder.predict(z)

## SAE

In [ ]:
decoded_imgs = autoencoder.predict(X_test2)

## Plots & Analysis

In [ ]:
def plotting(tested, predicted, epochs):
    %matplotlib inline
    n = 4
    plt.figure(figsize=(30,20))
    for i in range(1,n+1):
        # Original
        ax = plt.subplot(2, n, i)
        plt.imshow(tested[i,:,:,0], cmap = 'Blues')
        plt.colorbar()

        # Reconstructed
        ax = plt.subplot(2, n, i+n)
        plt.imshow(predicted[i,:,:,0], cmap = 'Blues')
        plt.colorbar()
    plt.show()

In [ ]:
print("              100 epochs: Trained on original")
plotting(X_test2, decoded_imgs[0], num_epoch)
print("              100 epochs: Trained on transformed")
plotting(X_test2, decoded_imgs[1], num_epoch)

In [ ]:
print(np.count_nonzero(X_test2), np.count_nonzero(X_test2)/(X_test2.shape[0]*X_test2.shape[1]*X_test2.shape[2])*100)
print(np.count_nonzero(decoded_imgs[.]),  np.count_nonzero(decoded_imgs[.])/(decoded_imgs[.].shape[0]*decoded_imgs[.].shape[1]*decoded_imgs[.].shape[2])*100)

In [ ]:
zval_org = X_test2.flatten()
zval_pred = [decoded_imgs[0].flatten(), decoded_imgs[1].flatten()]

In [ ]:
def statistics(zvalues):
    median = np.median(zvalues)
    upper_quartile = np.percentile(zvalues, 75)
    lower_quartile = np.percentile(zvalues, 25)

    iqr = upper_quartile - lower_quartile
    upper_whisker = zvalues[zvalues <= upper_quartile+1.5*iqr].max()
    upper_whisker2 = zvalues[zvalues <= upper_quartile+3*iqr].max()
    lower_whisker = zvalues[zvalues >= lower_quartile-1.5*iqr].min()
    lower_whisker2 = zvalues[zvalues >= lower_quartile-3*iqr].min()
    print(lower_whisker2, lower_whisker, lower_quartile, median, upper_quartile, upper_whisker, upper_whisker2)

In [ ]:
for i in range(len(zval_pred)):
    zval = zval_pred[i]
    statistics(zval)

In [ ]:
zval = zval_pred[#]
bound = ###
print(zval[zval < bound].shape[0]/zval.shape[0] * 100)

In [ ]:
for i in range(len(zval_pred)):
    print(mean_squared_error(zval_org, zval_pred[i]), math.sqrt(mean_squared_error(zval_org, zval_pred[i])))
    print(mean_squared_log_error(zval_org, zval_pred[i]), math.sqrt(mean_squared_log_error(zval_org, zval_pred[i])))

In [ ]:
%matplotlib inline
colors = ['b', 'r', 'g']

print("64 X 64 IMAGES, 100 EPOCHS: blue are original z's")
print()

zvalp1 = zval_pred[#]

zvalo_log = np.log10(zval_org[zval_org != 0])
zvalp_log = [np.log10(zvalp1[zvalp1 != 0]),np.log10(zvalp2[zvalp2 != 0])]

plt.figure(figsize=(10,5))
plt.hist(zvalo_log, bins = 50, color = colors[0], histtype = 'step', label = 'original')
plt.hist(zvalp_log[.], bins = 50, color = colors[1], histtype = 'step', label = 'predicted_org')
plt.legend()
plt.semilogy()
plt.xlabel("Log base 10 z-value", fontsize = 15)
plt.ylabel("Count (log base 10 scale)", fontsize = 15)
#plt.savefig("")
plt.show()

In [ ]:
phis = np.concatenate([np.nonzero(x)[0] for x in decoded_imgs[0]])
etas = np.concatenate([np.nonzero(x)[1] for x in decoded_imgs[0]])

In [ ]:
%matplotlib inline
num = 50
colors = ['b', 'r', 'g', 'c','m', 'y']
plt.hist(etas[.], bins = num, color = colors[3], histtype = 'step', label = 'rapp_org')
plt.hist(phis[.], bins = num, color = colors[4], histtype = 'step', label = 'phip_org')
plt.hist(rapidity, bins = num, color = colors[1], histtype = 'step', label = 'rap_org')
plt.hist(phi, bins = num, color = colors[0], histtype = 'step', label = 'phi_org')
plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left")
plt.semilogy()
plt.xlabel("Values", fontsize = 15)
plt.ylabel("Count (log base 10 scale)", fontsize = 15)
plt.show()